In [34]:
import torch
import numpy as np
import os
import pandas as pd

In [35]:
rating_file ='./data/ml-20m/'+'ratings.csv'

In [36]:
rating_pd = pd.read_csv(rating_file)

In [37]:
rating_pd.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [38]:
rating_pd .to_csv('./data/ratings.txt', sep='\t', index=False, header=False)

In [54]:
rating_np = np.loadtxt('./data/ratings.txt', delimiter='\t',dtype=np.int64)

C:\Users\24717\AppData\Local\Temp\ipykernel_23168\3252501746.py:1: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  rating_np = np.loadtxt('./data/ratings.txt', delimiter='\t',dtype=np.int64)


In [97]:
print(rating_np[:40])

[[         1          2          3 1112486027]
 [         1         29          3 1112484676]
 [         1         32          3 1112484819]
 [         1         47          3 1112484727]
 [         1         50          3 1112484580]
 [         1        112          3 1094785740]
 [         1        151          4 1094785734]
 [         1        223          4 1112485573]
 [         1        253          4 1112484940]
 [         1        260          4 1112484826]
 [         1        293          4 1112484703]
 [         1        296          4 1112484767]
 [         1        318          4 1112484798]
 [         1        337          3 1094785709]
 [         1        367          3 1112485980]
 [         1        541          4 1112484603]
 [         1        589          3 1112485557]
 [         1        593          3 1112484661]
 [         1        653          3 1094785691]
 [         1        919          3 1094785621]
 [         1        924          3 1094785598]
 [         1 

In [60]:
def classify_ratings(data):
    # 将数据转换为DataFrame以便于处理
    df = pd.DataFrame(data, columns=['UserID', 'MovieID', 'Rating', 'Timestamp'])

    # 计算每个用户的平均评分
    user_avg_ratings = df.groupby('UserID')['Rating'].mean().to_dict()

    # 初始化正样本和负样本列表
    positive_samples = []
    negative_samples = []

    # 遍历数据，分类正负样本
    for index, row in df.iterrows():
        user_id = row['UserID']
        rating = row['Rating']
        avg_rating = user_avg_ratings[user_id]

        # 判断是正样本还是负样本
        if rating > avg_rating:
            positive_samples.append([user_id, row['MovieID'], rating, row['Timestamp']])
        else:
            negative_samples.append([user_id, row['MovieID'], rating, row['Timestamp']])

    return positive_samples, negative_samples

In [61]:
positive_samples, negative_samples = classify_ratings(rating_np)

In [101]:
max_user_id = max(rating[0] for rating in rating_np)
max_movie_id = max(rating[1] for rating in rating_np)

In [64]:
print(max_user_id, max_movie_id)

138493 131262


In [66]:
print(rating_np.shape)

(20000263, 4)


In [72]:
num_users = max_user_id 
num_movies = max_movie_id 
embedding_dim = 50  # 嵌入向量的维度为50

In [73]:
def initialize_embeddings(num_users, num_movies, embedding_dim, seed=42):
    """
    初始化用户和电影的嵌入向量。
    
    参数:
    num_users (int): 用户数量。
    num_movies (int): 电影数量。
    embedding_dim (int): 嵌入向量的维度。
    seed (int): 随机种子，用于复现结果。
    
    返回:
    user_embeddings (numpy.ndarray): 用户嵌入向量。
    movie_embeddings (numpy.ndarray): 电影嵌入向量。
    """
    np.random.seed(seed)  # 设置随机种子
    # 使用正态分布初始化用户嵌入向量
    user_embeddings = np.random.normal(scale=0.1, size=(num_users, embedding_dim))
    
    # 使用正态分布初始化电影嵌入向量
    movie_embeddings = np.random.normal(scale=0.1, size=(num_movies, embedding_dim))
    
    return user_embeddings, movie_embeddings


In [74]:
user_embeddings, movie_embeddings = initialize_embeddings(num_users, num_movies, embedding_dim)

In [75]:
print("User Embeddings Shape:", user_embeddings.shape)
print("Movie Embeddings Shape:", movie_embeddings.shape)

User Embeddings Shape: (138493, 50)
Movie Embeddings Shape: (131262, 50)


In [79]:
print(user_embeddings[0])

[ 0.04967142 -0.01382643  0.06476885  0.15230299 -0.02341534 -0.0234137
  0.15792128  0.07674347 -0.04694744  0.054256   -0.04634177 -0.04657298
  0.02419623 -0.19132802 -0.17249178 -0.05622875 -0.10128311  0.03142473
 -0.09080241 -0.14123037  0.14656488 -0.02257763  0.00675282 -0.14247482
 -0.05443827  0.01109226 -0.11509936  0.0375698  -0.06006387 -0.02916937
 -0.06017066  0.18522782 -0.00134972 -0.10577109  0.08225449 -0.12208436
  0.02088636 -0.19596701 -0.1328186   0.01968612  0.07384666  0.01713683
 -0.01156483 -0.03011037 -0.1478522  -0.07198442 -0.04606388  0.10571222
  0.03436183 -0.17630402]


In [87]:
########################################划分数据集#####################################
def dataset_split(rating_np, ratio):
    print('splitting dataset ...')

    # train:eval:test = 6:2:2
    eval_ratio = 0.2
    test_ratio = 0.2
    n_ratings = rating_np.shape[0]

    eval_indices = np.random.choice(list(range(n_ratings)), size=int(n_ratings * eval_ratio), replace=False)
    left = set(range(n_ratings)) - set(eval_indices)
    test_indices = np.random.choice(list(left), size=int(n_ratings * test_ratio), replace=False)
    train_indices = list(left - set(test_indices))
    if ratio < 1:
        train_indices = np.random.choice(list(train_indices), size=int(len(train_indices) * ratio), replace=False)

    train_data = rating_np[train_indices]
    eval_data = rating_np[eval_indices]
    test_data = rating_np[test_indices]

    return train_data, eval_data, test_data

In [89]:
ratio = 0.8
train_data, eval_data, test_data = dataset_split(rating_np, ratio)

splitting dataset ...


In [90]:
max_user_id = max(rating[0] for rating in train_data)
max_movie_id = max(rating[1] for rating in train_data)

In [92]:
print(train_data.shape)

(9600127, 4)


[     81462       4831          0 1092110978]


In [105]:
np.save('./data/data_mid/positive_samples.npy', positive_samples)

In [106]:
np.save('negative_samples.npy', negative_samples)

In [107]:
print(train_data.shape)

(9600127, 4)
